In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
crime = pd.read_csv('2020_crime.csv')

In [ ]:
# Create our features
X = pd.get_dummies(df, columns=['']).drop(columns=[''], axis=1)

# Create our target
y = df[['']]

In [ ]:
# Split data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Naives Random

In [ ]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled_ros, y_resampled_ros = ros.fit_resample(X_train, y_train)

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model_ros = LogisticRegression(solver='lbfgs', random_state=1)
model_ros.fit(X_resampled_ros, y_resampled_ros)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred_ros = model_ros.predict(X_test)
balanced_accuracy_score(y_test, y_pred_ros)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred_ros)

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred_ros))

### SMOTE

In [ ]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=1, sampling_strategy='auto')
X_resampled_smote, y_resampled_smote = smote.fit_resample(X_train, y_train)

In [ ]:
# Train the Logistic Regression model using the resampled data
model_smote = LogisticRegression(solver='lbfgs', random_state=1)
model_smote.fit(X_resampled_smote, y_resampled_smote)

In [ ]:
# Calculated the balanced accuracy score
y_pred_smote = model_smote.predict(X_test)
balanced_accuracy_score(y_test, y_pred_smote)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_smote)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smote))

### Cluster Centroids

In [ ]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled_cc, y_resampled_cc = cc.fit_resample(X_train, y_train)

In [ ]:
# Train the Logistic Regression model using the resampled data
model_cc = LogisticRegression(solver='lbfgs', random_state=1)
model_cc.fit(X_resampled_cc, y_resampled_cc)

In [ ]:
# Calculated the balanced accuracy score
y_pred_cc = model_cc.predict(X_test)
balanced_accuracy_score(y_test, y_pred_cc)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_cc)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_cc))

### Combination (Over and Under) Sampling

In [ ]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
# YOUR CODE HERE
from imblearn.combine import SMOTEENN
smoteenn = SMOTEENN(random_state=0)
X_resampled_smoteenn, y_resampled_smoteenn = smoteenn.fit_resample(X, y)

In [ ]:
# Train the Logistic Regression model using the resampled data
model_smoteenn = LogisticRegression(solver='lbfgs', random_state=1)
model_smoteenn.fit(X_resampled_smoteenn, y_resampled_smoteenn)

In [ ]:
# Calculated the balanced accuracy score
y_pred_smoteenn = model_smoteenn.predict(X_test)
balanced_accuracy_score(y_test, y_pred_smoteenn)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_smoteenn)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_smoteenn))

### Balanced Random Forest

In [ ]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brfc_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brfc_model.fit(X_train, y_train)

In [ ]:
# Calculated the balanced accuracy score
y_pred_brfc = brfc_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred_brfc)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_brfc)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_brfc))

In [ ]:
# List the features sorted in descending order by feature importance
importances = brfc_model.feature_importances_
sorted(zip(brfc_model.feature_importances_, X.columns), reverse=True)

### Easy Ensemble AdaBoost Classifier

In [ ]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
ee_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
ee_model.fit(X_train, y_train)
# Train the EasyEnsembleClassifier

In [ ]:
# Calculated the balanced accuracy score
y_pred_ee = ee_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred_ee)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_ee)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_ee))

In [ ]:
# List the features sorted in descending order by feature importance
importances = brfc_model.feature_importances_
sorted(zip(ee_model.feature_importances_, X.columns), reverse=True)